## Classifying CIFAR-10 with Data Augmentation

In this exercise, we revisit CIFAR-10 and the networks we previously built.  We will use real-time data augmentation to try to improve our results.

When you are done going through the notebook, experiment with different data augmentation parameters and see if they help (or hurt!) the performance of your classifier.

In [14]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [19]:
num_classes = 10
y_test_lab=np.copy(y_test)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_test_lab

array([[3],
       [8],
       [8],
       ..., 
       [5],
       [1],
       [7]])

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In Exercise 6, we built two models.  One was smaller (181K parameters) while the second was larger (1.25M) parameters.  Below we use the smaller model and train it with data augmentation.

In [5]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          25632     
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
__________

We still have 181K parameters, even though this is a "small" model.


In [6]:
batch_size = 32

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Here we define the `ImageDataGenerator` that we will use to serve images to our model during the training process.  Currently, it is configured to do some shifting and horizontal flipping.

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
hist1=model_1.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=15,
                    validation_data=(x_test, y_test))

Epoch 1/15
1562/1562 [==============================] - 44s - loss: 1.7834 - acc: 0.3449 - val_loss: 1.4601 - val_acc: 0.4763
Epoch 2/15
1562/1562 [==============================] - 47s - loss: 1.5237 - acc: 0.4493 - val_loss: 1.4449 - val_acc: 0.4970
Epoch 3/15
1562/1562 [==============================] - 43s - loss: 1.4317 - acc: 0.4881 - val_loss: 1.2385 - val_acc: 0.5473
Epoch 4/15
1562/1562 [==============================] - 40s - loss: 1.3667 - acc: 0.5099 - val_loss: 1.1494 - val_acc: 0.5901
Epoch 5/15
1562/1562 [==============================] - 43s - loss: 1.3299 - acc: 0.5284 - val_loss: 1.2370 - val_acc: 0.5744
Epoch 6/15
1562/1562 [==============================] - 40s - loss: 1.2960 - acc: 0.5396 - val_loss: 1.1008 - val_acc: 0.6147
Epoch 7/15
1562/1562 [==============================] - 39s - loss: 1.2737 - acc: 0.5478 - val_loss: 1.0697 - val_acc: 0.6231
Epoch 8/15
1562/1562 [==============================] - 43s - loss: 1.2604 - acc: 0.5554 - val_loss: 1.1763 - val_acc:

How does the performance compare with the non-augmented training?

In [20]:
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
y_pred_class = model_1.predict_classes(x_test)


print('accuracy is {:.3f}'.format(accuracy_score(y_test_lab,y_pred_class)))

 9792/10000 [============================>.] - ETA: 0saccuracy is 0.394


## Exercise
### Your Turn

1. Experiment above with different settings of the data augmentation parameters.  Can you make the model do better?  Can you make it do worse?

2. As in Exercise 6, Build a more complicated model with the following pattern:
    - Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification
    - Use strides of 1 for all convolutional layers.

3. Use data augmentation to train this model.  Can you get better performance?

In [8]:
# Let's build a CNN using Keras' Sequential capabilities

model_2 = Sequential()

model_2.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_2.add(Activation('relu'))
model_2.add(Conv2D(32, (3, 3)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Conv2D(64, (3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv2D(64, (3, 3)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

In [9]:
## Check number of parameters

model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [10]:
# initiate RMSprop optimizer
opt_2 = keras.optimizers.rmsprop(lr=0.0005)

# Let's train the model using RMSprop
model_2.compile(loss='categorical_crossentropy',
              optimizer=opt_2,
              metrics=['accuracy'])

In [11]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model_2.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=10,
                    validation_data=(x_test, y_test))

Epoch 1/10
1562/1562 [==============================] - 285s - loss: 1.6458 - acc: 0.4004 - val_loss: 1.2687 - val_acc: 0.5424
Epoch 2/10
1562/1562 [==============================] - 286s - loss: 1.2839 - acc: 0.5421 - val_loss: 1.1052 - val_acc: 0.6133
Epoch 3/10
1562/1562 [==============================] - 285s - loss: 1.1442 - acc: 0.5981 - val_loss: 0.9260 - val_acc: 0.6776
Epoch 4/10
1562/1562 [==============================] - 286s - loss: 1.0669 - acc: 0.6270 - val_loss: 0.9117 - val_acc: 0.6869
Epoch 5/10
1562/1562 [==============================] - 278s - loss: 1.0191 - acc: 0.6475 - val_loss: 0.8498 - val_acc: 0.7112
Epoch 6/10
1562/1562 [==============================] - 278s - loss: 0.9991 - acc: 0.6562 - val_loss: 0.8147 - val_acc: 0.7184
Epoch 7/10
1562/1562 [==============================] - 278s - loss: 0.9880 - acc: 0.6628 - val_loss: 0.8305 - val_acc: 0.7183
Epoch 8/10
1562/1562 [==============================] - 279s - loss: 0.9812 - acc: 0.6659 - val_loss: 0.8807 - 